# Analytical formulas for the design of a superconducting dipole

Formulas retrieved from the course "Masterclass - Design of superconducting magnets for particle accelerators" from Ezio Todesco (CERN) 
<a href="https://indico.cern.ch/category/12408/">Indico page</a>

<br>
Our targets:

- Dipole magnet with 
- B1: 12 T
- b3	< 10
- b5	< 10
- b7	< 15
- b9	< 5
- b11	< 5

<br>
Cable parameters:

- Operating current < 20 kA
- Number of strands	< 40
- Strand diameter	< 1.1 mm
- Strand twist pitch	< 110 mm
- Cable width	< 21 mm
- Fiberglass insulation thickness	~ 0.15 mm
- Specified non-Cu Jc 	2450 A/mm2 (4.2 K, 12 T)
- Cu/non-Cu ratio	~ 1

In [2]:
import os

from scipy.optimize import minimize
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import plotly.express as px
import plotly.graph_objects as go

import sympy as sy
import IPython.display as disp

#### Field armonics
$$ B_y+iB_x = 10^{-4} \, B_1 \sum_{n=1}^{\infty}(b_n+ia_n)\left( \dfrac{x+iy}{R_{ref}}\right)^{n-1}$$
$a_n$ and $b_n$ are the normalized multipoles.

#### Harmonics of a single conductor
If a conductor carrying current $I$ is placed at $z_0=x_0+iy_0$ we can compute the $B_1$ component as:

$$ B_1 = - \dfrac{I \mu_0}{2 \pi}\cdot\dfrac{x_0}{x_0^2+y_0^2}$$

the contribution to the multipoles is:

<br>
$$b_n+ia_n = - \dfrac{I\mu_0 \, 10^4}{2\pi z_0 B_1} \left( \dfrac{R_{ref}}{z_0}\right)^{n-1} $$

#### Approximation of sector coils for dipoles
If a coil presents the an up-down symmetry we suppress the skew multipoles $a_n$. <br> If it presents a left-right antisymmetry, we suppress all the even number normal multipoles $b_{2m}$. <br> We end up having only \a specific set of allowed multipoles:

- B_1: dipole
- b_3: sextupole
- b_5: decapole
- b_7: ...

If a sector coil is defined by $$r \in \{R, R+w\}$$ $$\phi_c \in \{\phi_1, \phi_2\}$$

we consider there is a symmetric fraction below the midplane $\phi_c \in \{-\phi_2, -\phi_1\}$

The contribution to the multipole is:
$$ C_n = -\dfrac{j\mu_0R_{ref}^{n-1}}{\pi}\int_R^{R+w}\rho^{1-n}d\rho \cdot \left(\int_{-\phi_2}^{-\phi_1} \exp(-in\theta) d\theta + \int_{\phi_1}^{\phi_2} \exp(-in\theta) d\theta\right)$$

which is:

$$ B_1 = -\dfrac{2J\mu_0}{\pi}w (\sin (\phi_2)-\sin(\phi_1))$$

$$ B_3 = \dfrac{\mu_0 J R_{ref}^2}{\pi} \dfrac{\sin (3\phi_2)-\sin(3\phi_1)}{3} \left(\dfrac{1}{R}-\dfrac{1}{R+w}\right)$$ 

$$ B_5 = \dfrac{\mu_0 J R_{ref}^4}{\pi} \dfrac{\sin (5\phi_2)-\sin(5\phi_1)}{5} \left(\dfrac{1}{R^3}-\dfrac{1}{(R+w)^3}\right)$$ 

In [9]:
from sympy import *
x1, x2, y1, y2 = symbols("x1 x2 y1 y2", real=True)  
x = x1 + I*x2
y = y1 + I*y2
disp.display(x)

x1 + I*x2

In [16]:
mm = 1e-3
RR = 26*mm
RRref = 2/3*RR
ww = 20*mm
JJ = -400/(mm**2)
B1t = 12
B3t = 0
mmu0 = 4*np.pi*1e-7

J, mu0, pi, w, alpha, R, Rref, n, phi1, phi2, rho, theta = sy.symbols('J mu_0 pi w alpha R R_ref n phi_1 phi_2 rho theta', real=True)

C_n = -2*(J*mu0*Rref**(n-1))/pi*sy.integrate(rho**(1-n),(rho, R, R+w))*sy.integrate(exp(-I*n*theta),(theta, phi1, phi2))
B_n = sy.re(C_n)

B1 = B_n.subs({n:1})
B3 = B_n.subs({n:3})
B5 = B_n.subs({n:5})
B7 = B_n.subs({n:7})
B9 = B_n.subs({n:9})
disp.display(B1)
disp.display(B3)
disp.display(B5)
disp.display(B7)
disp.display(B9)

-2*J*mu_0*w*(-sin(phi_1) + sin(phi_2))/pi

-2*J*R_ref**2*mu_0*(-1/(R + w) + 1/R)*(-sin(3*phi_1)/3 + sin(3*phi_2)/3)/pi

-2*J*R_ref**4*mu_0*(-1/(3*(R + w)**3) + 1/(3*R**3))*(-sin(5*phi_1)/5 + sin(5*phi_2)/5)/pi

-2*J*R_ref**6*mu_0*(-1/(5*(R + w)**5) + 1/(5*R**5))*(-sin(7*phi_1)/7 + sin(7*phi_2)/7)/pi

-2*J*R_ref**8*mu_0*(-1/(7*(R + w)**7) + 1/(7*R**7))*(-sin(9*phi_1)/9 + sin(9*phi_2)/9)/pi

## Example:

In [ ]:
alpha1, alpha2, alpha3 = sy.symbols('alpha_1 alpha_2 alpha_3')

B01 = B1.subs({mu0:mmu0, w:ww, pi:np.pi, R:RR, Rref:RRref})
B03 = B3.subs({mu0:mmu0, w:ww, pi:np.pi, R:RR, Rref:RRref})
B05 = B5.subs({mu0:mmu0, w:ww, pi:np.pi, R:RR, Rref:RRref})
B07 = B7.subs({mu0:mmu0, w:ww, pi:np.pi, R:RR, Rref:RRref})

expr1 = B01.subs({phi1:0, phi2:alpha1}) + B01.subs({phi1:alpha2, phi2:alpha3})
expr2 = B03.subs({phi1:0, phi2:alpha1}) + B03.subs({phi1:alpha2, phi2:alpha3})
expr3 = B05.subs({phi1:0, phi2:alpha1}) + B05.subs({phi1:alpha2, phi2:alpha3})
expr4 = B07.subs({phi1:0, phi2:alpha1}) + B07.subs({phi1:alpha2, phi2:alpha3})

eq1 = sy.Eq(expr1,12)
eq2 = sy.Eq(expr2,0)
eq3 = sy.Eq(expr3,0)
eq4 = sy.Eq(expr4,0)

[(JJ, a1, a2, a3)] = sy.solve([eq1,eq2, eq3, eq4],(J, alpha1, alpha2, alpha3))

# print("Current density J is: ", JJ*mm*mm, "A/mm2")
# print("Angle of the sector is: ", aa*180/np.pi, " deg")

#### Example: Single-sector coil
Find the angle $\alpha$ and current density $j$ that generate B1 = 12 T,  and B_3 = 0, inside an aperture of R = 26 mm with a coil width of 20 mm. Take Rref = 2/3 R

In [2]:
B1 = -2*J*mu0/pi*w*sy.sin(alpha)
B3 = J*mu0*Rref**2/pi*sy.sin(3*alpha)/3*(1/R-1/(R+w))

print("B1: ")
disp.display(B1)
print("B3: ")
disp.display(B3)

B1: 


-2*J*mu_0*w*sin(alpha)/pi

B3: 


J*R_ref**2*mu_0*(-1/(R + w) + 1/R)*sin(3*alpha)/(3*pi)

In [3]:
expr1 = B1.subs({mu0:mmu0, w:ww, pi:np.pi})
expr2 = B3.subs({mu0:mmu0, w:ww, pi:np.pi, Rref:RRref, R:RR})
eq1 = sy.Eq(expr1,12)
eq2 = sy.Eq(expr2,0)
disp.display(eq1)
disp.display(eq2)

Eq(-1.6e-8*J*sin(alpha), 12)

Eq(6.69887278582931e-10*J*sin(3*alpha), 0)

In [4]:
[(JJ, aa)] = sy.solve([eq1,eq2],(J, alpha))

print("Current density J is: ", JJ*mm*mm, "A/mm2")
print("Angle of the sector is: ", aa*180/np.pi, " deg")

Current density J is:  -866.025403784439 A/mm2
Angle of the sector is:  60.0000000000000  deg


### Now two layer coil 


In [1]:
expr = pi + 1
expr.subs(pi, np.pi)
B1
Rref
B3
expr1
expr2

NameError: name 'pi' is not defined